# Cybershuttle SDK -  Molecular Dynamics
> Define, run, monitor, and analyze molecular dynamics experiments in a HPC-agnostic way.

This notebook shows how users can setup and launch a **NAMD** experiment with replicas, monitor its execution, and run analyses both during and after execution.

## Installing Required Packages

First, install the `airavata-python-sdk-test` package from the pip repository.

In [ ]:
%pip install -e airavata-api/airavata-client-sdks/airavata-python-sdk

## Importing the SDK

In [ ]:
%cd airavata-api/airavata-client-sdks/airavata-python-sdk/samples
import airavata_experiments as ae
from airavata_experiments import md

## Authenticating

To authenticate for remote execution, call the `ae.login()` method.
This method will prompt you to enter your credentials and authenticate your session.

In [ ]:
ae.login()

Once authenticated, the `ae.list_runtimes()` function can be called to list HPC resources that the user has access to.

In [ ]:
runtimes = ae.list_runtimes()
ae.display(runtimes)

## Uploading Experiment Files

Drag and drop experiment files onto the workspace that this notebook is run on.

```bash
(sh) $: tree data
data
├── b4pull.pdb
├── b4pull.restart.coor
├── b4pull.restart.vel
├── b4pull.restart.xsc
├── par_all36_water.prm
├── par_all36m_prot.prm
├── pull.conf
├── structure.pdb
└── structure.psf

1 directory, 9 files

```

## Defining a NAMD Experiment

The `md.NAMD.initialize()` is used to define a NAMD experiment.
Here, provide the paths to the `.conf` file, the `.pdb` file, the `.psf` file, any optional files you want to run NAMD on.
You can preview the function definition through auto-completion.

```python
def initialize(
    name: str,
    config_file: str,
    pdb_file: str,
    psf_file: str,
    ffp_files: list[str],
    other_files: list[str] = [],
    parallelism: Literal['CPU', 'GPU'] = "CPU",
    num_replicas: int = 1
) -> Experiment[ExperimentApp]
```

To add replica runs, simply call the `exp.add_replica()` function.
You can call the `add_replica()` function as many times as you want replicas.
Any optional resource constraint can be provided here.

You can also call `ae.display()` to pretty-print the experiment.

In [ ]:
exp = md.NAMD.initialize(
    name="yasith_namd_experiment",
    config_file="data/pull.conf",
    pdb_file="data/structure.pdb",
    psf_file="data/structure.psf",
    ffp_files=[
      "data/par_all36_water.prm",
      "data/par_all36m_prot.prm"
    ],
    other_files=[
      "data/b4pull.pdb",
      "data/b4pull.restart.coor",
      "data/b4pull.restart.vel",
      "data/b4pull.restart.xsc",
    ],
    parallelism="GPU",
)
exp.add_replica()
ae.display(exp)

## Creating an Execution Plan

Call the `exp.plan()` function to transform the experiment definition + replicas into a stateful execution plan.

In [ ]:
plan = exp.plan()
ae.display(plan)

## Saving the Plan

A created plan can be saved locally (in JSON) or remotely (in a user-local DB) for later reference.

In [ ]:
plan.save()  # this will save the plan in DB
plan.save_json("plan.json")  # save the plan state locally

## Launching the Plan

A created plan can be launched using the `plan.launch()` function.
Changes to plan states will be automatically saved onto the remote.
However, plan state can also be tracked locally by invoking `plan.save_json()`.

In [ ]:
plan.launch()
plan.save_json("plan.json")

## Loading a Saved Plan

A saved plan can be loaded by calling `ae.plan.load_json(plan_path)` (for local plans) or `ae.plan.load(plan_id)` (for remote plans).

In [ ]:
plan = ae.plan.load_json("plan.json")
plan = ae.plan.load(plan.id)
ae.display(plan)

## Fetching User-Defined Plans

The `ae.plan.query()` function retrieves all plans stored in the remote.

In [ ]:
plans = ae.plan.query()
ae.display(plans)

## Checking Plan Status

The plan's execution status can be checked by calling `plan.status()`.

In [ ]:
plan.status()

## Managing Plan Execution

The `plan.stop()` function will stop a currently executing plan.
The `plan.join()` function would block code execution until the plan completes its execution.

In [ ]:
plan.stop()
plan.join()

## Running File Operations

Displaying the status and files generated by each replica (task)

In [ ]:
for task in plan.tasks:
    status = task.status()
    print(status)
    # task.upload("data/sample.txt")
    files = task.ls()
    display(files)
    display(task.cat("NAMD.stderr"))
    # task.download("NAMD.stdout", "./results")
    task.download("NAMD_Repl_1.out", "./results")

Displaying the intermediate results generated by each replica (task)

In [ ]:
for index, task in enumerate(plan.tasks):

    @task.context(packages=["matplotlib", "pandas"])
    def analyze(x, y, index, num_tasks) -> None:
        from matplotlib import pyplot as plt
        import pandas as pd
        df = pd.read_csv("data.csv")
        plt.figure(figsize=(x, y))
        plt.plot(df["x"], df["y"], marker="o", linestyle="-", linewidth=2, markersize=6)
        plt.title(f"Plot for Replica {index} of {num_tasks}")

    analyze(3, 4, index+1, len(plan.tasks))